This script makes an S21 measure in a user specified frequency range, for each "triplet" of values for:

- DC voltage

- Pump power

- Pump frequency


extracted form user specified 3d lattice (3 intervals), in order to find the "optimal parameters" for the PARAMP

In [2]:
"""ESTABLISH A CONNECTION WITH THE 3 DEVICES
   -DC generator
   -AC generator (Pump)
   -VNA
"""

# DC generator
import ..src.sim
resource = 'ASRL34::INSTR'
instr_sim = sim.ManageSIM(resource)

# VNA
import ..src.vna
IP = "192.168.3.51"
instr_vna = vna.ManageVNA(IP)

# AC generator (Pump)
import ..src.sma
resource = 'TCPIP0::192.168.3.2::INSTR'
instr_sma = sma.ManageSMA(resource)

In [13]:
"""SET RANGES FOR PARAMETERS VARIATION"""

import numpy as np

# Pump frequency
freq_range = [11e9, 13e9] #[Hz]
# Pump output power
ampl_range = [-40, -20] #[dBm]
# DC voltage
volt_range = [0.4, 0.5] #[V]

# Number of equispaced sampled point in the ranges
n_freq = 5
n_ampl = 2
n_volt = 2
freqs = np.linspace(freq_range[0],freq_range[1], n_freq) 
ampls = np.linspace(ampl_range[0],ampl_range[1], n_ampl)
volts = np.linspace(volt_range[0],volt_range[1], n_volt)

# Set the VNA scan parameters
# [The scan is performed after each parameter variation]
fmin = 30e3 #[Hz]
fmax = 14e9 #[Hz]
npoints = 100
navgs = 2
powerdBm = -40 #[dBm]

# Turn ON DC and Pump output
instr_sma.set_output(1)
instr_sim.set_output(1)

In [14]:
"""CYCLE THE 3 PARAMETERS AND SAVE S21 MEASUREMENTS TO .h5 FILE"""
#NOTe: Before we MUST select manually the S21 port!!!


import time
import h5py
import sys

# Open a NEW .h5 file where measures will be stored
filename = "data.h5"    # CHANGE EACH TIME!
hf = h5py.File('data.h5', 'w')
# Check if the file already exists
if(len(hf.keys()) > 0):
    hf.close()
    sys.exit("Cannot create .h5 file with this name. A file named " + str(filename) + "already exists!")
# Define an index to enumerate measurements
# [necessary to recover values assigned to the parameters
# for each measurement]
index = 0
# Set ranges of the variyng parameters [necessary for recovering]
params_mat = [freqs, ampls, volts]
dataset_name = "params"
hf.create_dataset(dataset_name, data = params_mat)
# Define how many second wait between a query and
# the next one
query_time = 1 #[seconds]


# Cycle over all combinations of the 3 parameters
for volt in volts:
    time.sleep(query_time)
    instr_sim.set_voltage(volt) #Update DC voltage
    for ampl in ampls:
        time.sleep(query_time)
        instr_sma.set_amplitude(ampl)   #Update Pump power (amplitude)
        for freq in freqs:
            time.sleep(query_time)
            instr_sma.set_freq(freq)    #Update Pump frequency
    
            #Scan S21
            freq, I, Q = instr_vna.single_scan(fmin,fmax,powerdBm,npoints,navgs)
             
            #Write scan resuts to .h5 file
            #each measurement is a dataset made of 3 arrays: freq, I, Q
            #datasets are labeled by and index: "dataset_$index"
            Mat = [freq,I,Q]
            dataset_name = "measure_" + str(index)
            hf.create_dataset(dataset_name, data=Mat)
            index = index + 1   #Update index that labels measurements

hf.close()  #Close the file where we wrote the measurements

In [ ]:
"""READ MEASUREMENTS FROM FILE"""
#filename = "data.h5"
hf = h5py.File(filename, 'r')
index = 3
dataset_name = "dataset_" + str(index)
val = hf.get(dataset_name)